In [1]:
#pip install pandas
#pip install seaborn
#pip install scikit-learn

# import pandas as pd
# import numpy as np
# # import seaborn as sns 
# # import matplotlib.pyplot as plt
# # from statannotations.Annotator import Annotator
# from sklearn.preprocessing import StandardScalar

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d  # noqa: F401
import numpy as np
from sklearn import datasets, decomposition
import pandas as pd

In [7]:
#pandas, sklearn, etc. 


In [14]:
#load in csv

df = pd.read_csv('/Users/kristinachan/Analysis/2024-08-02_longcovid_interstate/LC_household_pulse_for_BV_8.csv')
date_columns = df.filter(like="/2")
date_columns = date_columns.columns.tolist()
# list = df_dates.columns
# list
#option to make the index the state, may be more useful as the column for now

In [21]:
df.columns

Index(['State', 'Jun 24 Value', 'Jun 24 Low CI', 'Jun 24 High CI', 'Urban',
       'Rural', 'Conservative', 'Moderate', 'Liberal', 'Political Don't know',
       'Political sum', 'Below Poverty Line', 'White', 'Black', 'AI_AN',
       'Asian', 'NHPI', 'Hispanic', 'Multiple',
       'Percent of total pop with at least one dose',
       'Percent of pop with an updated (bivalent) booster dose',
       'Percent of 5+ pop with a completed primary series',
       'Percent of 12+ pop with at least one dose',
       'Percent of 18+ pop with a completed primary series',
       'Percent of 65+ pop with a completed primary series', 'Paid sick leave',
       'Eviction moratoria duration', '7/11/22', '8/8/22', '9/26/22',
       '10/17/22', '11/14/22', '12/19/22', '1/16/23', '2/13/23', '3/13/23',
       '4/10/23', '5/8/23', '6/19/23', '7/10/23', '8/7/23', '9/4/23',
       '10/2/23', '10/30/23', '2/5/24', '3/4/24', '4/1/24', '4/29/24',
       '5/27/24', '6/24/24', 'avg_long_covid_rate', 'rural_urban'

In [31]:
df = df.rename(columns={
    'State': 'state',
    'Avg': 'avg_long_covid_rate',
    'Paid sick leave': 'paid_sick_leave',
    'Below Poverty Line': 'below_poverty_line',
    'White': 'white',
    'Black': 'black',
    'AI_AN': 'ai_an', #american indian alaskan native
    'Asian': 'asian',
    'NHPI': 'nhpi', #native hawaiian and pacific islander
    'Hispanic': 'hispanic',
    'Multiple': 'multi_ethnic',
    'Eviction moratoria duration':'eviction_moratia_duration',
    'Percent of total pop with at least one dose': 'vax_min_one_dose',
    'Percent of pop with an updated (bivalent) booster dose': 'vax_bivalent_booster_dose',
    'Percent of 5+ pop with a completed primary series': 'vax_fiveup_primaryseries',
    'Percent of 12+ pop with at least one dose': 'vax_twelveup_min_one_dose',
    'Percent of 18+ pop with a completed primary series': 'vax_eighteenup_primaryseries',
    'Percent of 65+ pop with a completed primary series': 'vax_sixtyfiveup_primaryseries', 
    
})

df['rural_urban'] = df['Rural']
df['lib_conserv_scale'] = (df['Liberal'] + 0.5 * df['Moderate']) / (df['Conservative'] + df['Moderate'] + df['Liberal']) * 100
df['sick_leave'] = np.where(df['paid_sick_leave'] == 0, 'unpaid', 'paid')

df.head().filter(like='vax')


,vax_min_one_dose,vax_bivalent_booster_dose,vax_fiveup_primaryseries,vax_twelveup_min_one_dose,vax_eighteenup_primaryseries,vax_sixtyfiveup_primaryseries
0,65.1,7.7,56.6,74.2,63.3,85.3
1,73.2,14.2,69.8,82.9,75.9,89.5
2,78.4,15.8,70.2,86.9,75.2,90.8
3,70.1,11.0,60.6,79.5,66.8,84.2
4,85.1,20.6,79.1,94.0,83.9,93.2


In [32]:
df.columns

Index(['state', 'Jun 24 Value', 'Jun 24 Low CI', 'Jun 24 High CI', 'Urban',
       'Rural', 'Conservative', 'Moderate', 'Liberal', 'Political Don't know',
       'Political sum', 'below_poverty_line', 'white', 'black', 'ai_an',
       'asian', 'nhpi', 'hispanic', 'multi_ethnic', 'vax_min_one_dose',
       'vax_bivalent_booster_dose', 'vax_fiveup_primaryseries',
       'vax_twelveup_min_one_dose', 'vax_eighteenup_primaryseries',
       'vax_sixtyfiveup_primaryseries', 'paid_sick_leave',
       'eviction_moratia_duration', '7/11/22', '8/8/22', '9/26/22', '10/17/22',
       '11/14/22', '12/19/22', '1/16/23', '2/13/23', '3/13/23', '4/10/23',
       '5/8/23', '6/19/23', '7/10/23', '8/7/23', '9/4/23', '10/2/23',
       '10/30/23', '2/5/24', '3/4/24', '4/1/24', '4/29/24', '5/27/24',
       '6/24/24', 'avg_long_covid_rate', 'rural_urban', 'lib_conserv_scale',
       'sick_leave'],
      dtype='object')

In [69]:
features = ['rural_urban', 'lib_conserv_scale',
       'below_poverty_line', 'white', 'black', 'ai_an',
       'asian', 'nhpi', 'hispanic', 'multi_ethnic', 'vax_min_one_dose',
       'vax_bivalent_booster_dose', 'vax_fiveup_primaryseries',
       'vax_twelveup_min_one_dose', 'vax_eighteenup_primaryseries',
       'vax_sixtyfiveup_primaryseries', 'paid_sick_leave',
       'eviction_moratia_duration',] #currently have 18 features

pca_df = df[['rural_urban', 'lib_conserv_scale',
       'below_poverty_line', 'white', 'black', 'ai_an',
       'asian', 'nhpi', 'hispanic', 'multi_ethnic', 'vax_min_one_dose',
       'vax_bivalent_booster_dose', 'vax_fiveup_primaryseries',
       'vax_twelveup_min_one_dose', 'vax_eighteenup_primaryseries',
       'vax_sixtyfiveup_primaryseries', 'paid_sick_leave',
       'eviction_moratia_duration', 
       'state', 'sick_leave', 'avg_long_covid_rate',]]

# HERE: option to create more pca_df columns that you want to color by later 

pca_df = pca_df.drop(df[df['state'] == 'United States'].index) #this column is for all of the united states and has some nan values
pca_df.head()


,rural_urban,lib_conserv_scale,below_poverty_line,white,black,ai_an,asian,nhpi,hispanic,multi_ethnic,...,vax_bivalent_booster_dose,vax_fiveup_primaryseries,vax_twelveup_min_one_dose,vax_eighteenup_primaryseries,vax_sixtyfiveup_primaryseries,paid_sick_leave,eviction_moratia_duration,state,sick_leave,avg_long_covid_rate
0,42.3,29.120879,16.5,0.647117,0.264309,0.005532,0.015477,0.000526,0.041489,0.025550,...,7.7,56.6,74.2,63.3,85.3,0.0,59.0,Alabama,unpaid,7.6
1,35.1,44.148936,11.4,0.587953,0.032379,0.148749,0.065232,0.015896,0.052876,0.096915,...,14.2,69.8,82.9,75.9,89.5,0.0,114.0,Alaska,unpaid,7.0
2,10.7,41.397849,12.6,0.529302,0.047562,0.037277,0.036369,0.001968,0.289346,0.058177,...,15.8,70.2,86.9,75.2,90.8,1.0,0.0,Arizona,paid,6.9
3,44.5,35.106383,16.8,0.709676,0.153314,0.007915,0.017788,0.004217,0.075664,0.031426,...,11.0,60.6,79.5,66.8,84.2,0.0,0.0,Arkansas,unpaid,7.3
4,5.8,48.936170,12.1,0.347300,0.055794,0.004067,0.155872,0.003738,0.360048,0.073181,...,20.6,79.1,94.0,83.9,93.2,1.0,884.0,California,paid,5.9


## PCA Analysis 

#### Normalize the feature data

In [73]:
from sklearn.preprocessing import StandardScaler

x = pca_df.loc[:, features].values #takes in the entire pca table, and then selects the features we care about for the pca analysis itself
x = StandardScaler().fit_transform(x) #normalize the values of the selected features
x.shape
np.mean(x), np.std(x) #check to see if normalized data normalized properly -- mean of zero, sd of one
feat_cols = ['feature'+str(i) for i in range(x.shape[1])] #change the name of the columns
normalized_pca_df = pd.DataFrame(x, columns=feat_cols)
normalized_pca_df.tail()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17
46,-0.174825,-0.497376,-0.598812,-0.416559,0.767528,-0.442661,0.483031,-0.210281,-0.228840,-0.013551,1.427802,2.486316,1.684654,1.044633,1.523789,0.805231,-0.676123,-0.528189
47,-0.694210,1.093873,-0.858564,-0.087172,-0.691591,-0.106194,1.052528,0.304374,0.091727,0.547859,1.085499,0.444587,0.847076,1.044633,0.812481,0.805231,1.479020,0.866340
48,1.889395,-1.425605,2.035816,1.528166,-0.745745,-0.459616,-0.707080,-0.248401,-0.963939,-0.678902,0.558169,1.227506,0.786528,0.948096,0.835062,0.805231,-0.676123,-0.693789
49,0.391170,-0.638268,-0.598812,0.841575,-0.485989,-0.221814,-0.276212,-0.235721,-0.454439,-0.514568,-1.107086,-0.983088,-0.979450,-1.296386,-1.039178,-1.460736,-0.676123,-0.682168
50,0.730768,-0.899560,-0.264845,1.025942,-0.995539,0.188160,-0.683436,-0.212358,-0.168892,-0.365322,-0.394727,0.736263,-0.061141,-0.391353,-0.056896,0.805231,-0.676123,-0.856484


In [74]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
principalComponents = pca.fit_transform(x)

In [ ]:
#with pandas dataframe 
metadata_columns = ['sepal length (cm)' , 'sepal width (cm)', 'petal length (cm)' , 'petal width (cm)']
color_by_column = 'state'

X = iris_df[metadata_columns]  #feature matrix (i.e. columns of interest)
y = iris_df[color_by_column] # target labels / categorizations /  value that you split by (i.e. the species of iris)

fig = plt.figure(1, figsize=(4,3))
plt.clf() #clear current figure

ax = fig.add_subplot(111, projection="3d", elev=48, azim=134)
ax.set_position([0,0,0.95,1])

plt.cla() #clear current axes

pca = decomposition.PCA(n_components=3)
pca.fit(X)
X = pca.transform(X)


#Adds text labels to the 3D plot for each Iris species
for name, label in [("Setosa",0), ("Versicolor",1), ("Verginica",2)]:
    ax.text3D(
        X[y == label, 0].mean(),
        X[y == label, 1].mean() + 1.5, #im confused why it's offset. is it so i can read it better?
        X[y == label, 2].mean(),
        name, #text label for the class8  
        horizontalalignment="center",
        bbox=dict(alpha=0.5, edgecolor="w", facecolor="w"),
    )

y = np.choose(y, [1,2,0]).astype(float)

# puts the scatter plot on the 3d plot
    # Scatter plot of the PCA-transformed data in 3D space
    # c=y specifies the colors based on the target labels
    # cmap=plt.cm.nipy_spectral specifies the colormap
    # edgecolor="k" specifies the edge color of the points
ax.scatter(X[:, 0], X[:,1], X[:,2], c=y, cmap=plt.cm.nipy_spectral, edgecolor="k")

ax.xaxis.set_ticklabels([]) #gets rid of the arbitrary number on the axes
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])

plt.show()

## Longitudinal Across Time: Grabbing Date Column Names, can plot on x-axis

In [50]:
df
df.filter(like='leave').head(3)


,Paid sick leave
0,0.0
1,0.0
2,1.0


In [51]:
date_columns

['7/11/22',
 '8/8/22',
 '9/26/22',
 '10/17/22',
 '11/14/22',
 '12/19/22',
 '1/16/23',
 '2/13/23',
 '3/13/23',
 '4/10/23',
 '5/8/23',
 '6/19/23',
 '7/10/23',
 '8/7/23',
 '9/4/23',
 '10/2/23',
 '10/30/23',
 '2/5/24',
 '3/4/24',
 '4/1/24',
 '4/29/24',
 '5/27/24',
 '6/24/24']

In [ ]:
#you want to average the values of 1) paid leave and 2)unpaid leave separately
#legend: color by -- subgroup of paid & unpaid
#x-axis: longitudinal values in increasing order
#y-axis: average rates of long covid
#points: plot the average value for that date and subgroup
#line graph 

#next steps after that: scatterplot with linear regression 
    #q: do i care about the correlation between time and increase/decreasing covid rank?... 
#can also visualize with boxplots to see if trends are more apparent

In [46]:
# df[date_columns] = df[date_columns].apply(pd.to_datetime)
# df[date_columns]
# Calculate the average rate of long covid for each date based on 'Paid sick leave' status
# avg_long_covid = df['Paid sick leave'].groupby('0')[date_columns].mean().T
# avg_long_covid.columns = ['Unpaid Leave', 'Paid Leave']

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# from statannotations.Annotator import Annotator
# import pandas as pd
# %matplotlib inline 

y = f'{scrubbed}alpha_faith_pd_{subset}_{subexp}'
# y = f'{scrubbed}alpha_observed_features_{subset}_{subexp}'
# y = f'{scrubbed}alpha_shannon_{subset}_{subexp}'
# y = f'{scrubbed}alpha_evenness_{subset}_{subexp}'
# y = f'{scrubbed}alpha_faith_pd_dap_control'
# # y = f'{scrubbed}alpha_shannon_dap_control'
# # y = f'{scrubbed}alpha_observed_features_dap_control'

x="mouse_age"
hue= "mouse_genotype_treatment"
hue_order=['WT_Vehicle_ctrl', 'WT_Dapensutrile', 'CtnsKO_Vehicle_ctrl', 'CtnsKO_Dapensutrile']
order=["2months", "4months", "9months"]


relevant_treatments = ['WT_Dapensutrile', 'WT_Vehicle_ctrl', 'CtnsKO_Dapensutrile', 'CtnsKO_Vehicle_ctrl']
# relevant_treatments = ['WT_Vehicle_ctrl', 'CtnsKO_Vehicle_ctrl']
# relevant_treatments = ['WT_Dapensutrile','CtnsKO_Dapensutrile']

relevant_treatments_df = metadata_df[metadata_df['mouse_genotype_treatment'].isin(relevant_treatments)]
# relevant_treatments_df['mouse_genotype_treatment'].value_counts()
# relevant_treatments_df.filter(like='alpha').head(30)

## ** SELECT Y AXIS METRIC YOU WANT TO VIEW: **
# alpha_metric_of_interest = 'alpha_observed_features_dap_control'
alpha_metric_of_interest = 'alpha_faith_pd_dap_control'
# alpha_metric_of_interest = 'alpha_shannon_dap_control'




for treatment in relevant_treatments:
    # for dot in custom_dot:
    #         for fill in custom_fill:
    lineplot = sns.lineplot(data=relevant_treatments_df[relevant_treatments_df['mouse_genotype_treatment'] == treatment], 
                x=x, y=y, label=treatment, #marker= custom_dot[dot], fillstyle=custom_fill[fill],
                color=custom_palette[treatment], linestyle=custom_linestyles[treatment],
                hue_order=hue_order
                # orient='h'
                           )

# got the stats from the boxplots above.. 

ax = sns.lineplot(data=relevant_treatments_df, x='mouse_age', y=y, hue='mouse_genotype_treatment', color=custom_colors[treatment]) #label='mouse_genotype_treatment', color=custom_colors)
                  
# , hue='mouse_genotype_treatment') #, errorbar=('ci', 95), err_style="band")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

plt.title(label=f'Alpha Diversity Over Time: {y}')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:


# Plotting
plt.figure(figsize=(12, 6))

# Plot the data points and lines
sns.lineplot(data=avg_long_covid, markers=True, dashes=False, palette=['brown', 'green'])

# Add horizontal lines for the average 'Avg' values
avg_paid = df[df['Paid sick leave'] == 1]['Avg'].mean()
avg_unpaid = df[df['Paid sick leave'] == 0]['Avg'].mean()
plt.axhline(avg_paid, linestyle='--', color='lightgreen', label='Avg Paid Leave')
plt.axhline(avg_unpaid, linestyle='--', color='lightbrown', label='Avg Unpaid Leave')

# Customize the legend
plt.legend(title='Categories', loc='upper left', labels=['Paid Leave', 'Unpaid Leave', 'Avg Paid Leave', 'Avg Unpaid Leave'])

# Set plot title and labels
plt.title('Average Rate of Long Covid Over Time')
plt.xlabel('Date')
plt.ylabel('Average Rate of Long Covid')
plt.xticks(rotation=45)

plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Sample data creation for demonstration (replace this with your actual DataFrame)
data = {
    'Paid sick leave': [0, 1, 0, 1, 0],
    '7/11/22': [10, 15, 20, 25, 30],
    '9/26/22': [12, 17, 22, 27, 32],
    '10/17/22': [14, 19, 24, 29, 34],
    '3/13/23': [16, 21, 26, 31, 36],
    '2/5/24': [18, 23, 28, 33, 38],
    'Avg': [14, 19, 24, 29, 34]
}

df = pd.DataFrame(data)

# Convert date columns to datetime
date_columns = ['7/11/22', '9/26/22', '10/17/22', '3/13/23', '2/5/24']
df[date_columns] = df[date_columns].apply(pd.to_datetime)

# Calculate the average rate of long covid for each date based on 'Paid sick leave' status
avg_long_covid = df.groupby('Paid sick leave')[date_columns].mean().T
avg_long_covid.columns = ['Unpaid Leave', 'Paid Leave']

# Plotting
plt.figure(figsize=(12, 6))

# Plot the data points and lines
sns.lineplot(data=avg_long_covid, markers=True, dashes=False, palette=['brown', 'green'])

# Add horizontal lines for the average 'Avg' values
avg_paid = df[df['Paid sick leave'] == 1]['Avg'].mean()
avg_unpaid = df[df['Paid sick leave'] == 0]['Avg'].mean()
plt.axhline(avg_paid, linestyle='--', color='lightgreen', label='Avg Paid Leave')
plt.axhline(avg_unpaid, linestyle='--', color='lightbrown', label='Avg Unpaid Leave')

# Customize the legend
plt.legend(title='Categories', loc='upper left', labels=['Paid Leave', 'Unpaid Leave', 'Avg Paid Leave', 'Avg Unpaid Leave'])

# Set plot title and labels
plt.title('Average Rate of Long Covid Over Time')
plt.xlabel('Date')
plt.ylabel('Average Rate of Long Covid')
plt.xticks(rotation=45)

plt.show()


In [ ]:
#scatterplots, etc